In [2]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [4]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os, os.path, sys

In [5]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\Y"

In [6]:
dir = os.listdir(path)

Ucitavanje fajlova

In [7]:
file_names = []
for file in dir:
    file_names.append(file)

In [8]:
percent_train = 0.8

In [9]:
percent_val = 0.2

In [10]:
train_files_names = file_names[:int(percent_train*len(file_names))]
#test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate koordinate

In [11]:
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\yCmd.txt",header=None).astype('int'))

In [12]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
# test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
#                               for file in test_file_names], axis=0)

In [13]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [14]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

Adekvatno rasporedjivanje podataka kako bi se izbeglo fiktivno stanje

In [15]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(61000):
        exec(f"razlika_trening_{j}.append(y_command[i] - train_[j*61000+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [16]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [16]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for i in range(61000):    
    for j in range(len(train_files_names)):
        exec(f'current_file = razlika_trening_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

# TESTIRANJE ARHITEKTURE i LR/BATCH

radjeno u vise iteracija sa razlicitim vrednostima zbog vremenske kompleksnosti

In [19]:
gru_unit = [8,16]
unit1 = [8,16]
unit2 = [8,16]
batch_size = [128,256]
time_steps_list = [8,10]

In [20]:
best_skor = float('-inf')
best_gru = None
best_unit1 = None
best_unit2 = None
best_batch = None
best_step = None
for step in time_steps_list:

    #delimo podatke na osnovu velicine buffera
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')

    train_X = np.asarray(train_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  

    #uzimamo percent_val sa kraja train_X skupa     
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
    
    for batch in batch_size:        
        for gru in gru_unit:
            for uni1 in unit1:
                for uni2 in unit2:

                    keras.utils.set_random_seed(7)
                    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                    model = Sequential()
                    model.add(GRU(units = gru, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
                    model.add(Dense(units = uni1, activation=keras.layers.LeakyReLU()))
                    model.add(Dense(units = uni2, activation=keras.layers.LeakyReLU()))
                    model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
                    model.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                    model.fit(train_X, train_y, epochs = 10, batch_size = batch, verbose = 2, shuffle = False)
                    test_predict_MSE = model.predict(val_x)
                    
                    #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                    #radimo evaluaciju na mean 3 sigma testu
                    i =(61000-step) - val_x.shape[0]%(61000-step) -1
                    mean_3std = 0
                    for j in range(len(test_predict_MSE)):
                        if i%61000 == 0:
                            i = step
                        if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                mean_3std += 1
                        i += 1


                    if mean_3std > best_skor:
                        best_skor = mean_3std
                        best_gru = gru
                        best_unit1 = uni1
                        best_unit2 = uni2
                        best_step = step
                        best_batch = batch

                        print('best_skor: ', best_skor,'best_gru: ', best_gru,'best_unit1: ', best_unit1,'best_unit2: ', best_unit2, 'best_batch', best_batch, 'best_step', best_step)
print('best_skor: ', best_skor)                    
print('best_gru: ', best_gru)
print('best_unit1: ', best_unit1)
print('best_unit2: ', best_unit2)
print('best_batch: ', best_batch)
print('best_step: ', best_step)

Epoch 1/10
9149/9149 - 44s - loss: 19.4844 - 44s/epoch - 5ms/step
Epoch 2/10
9149/9149 - 38s - loss: 3.4600 - 38s/epoch - 4ms/step
Epoch 3/10
9149/9149 - 38s - loss: 2.5302 - 38s/epoch - 4ms/step
Epoch 4/10
9149/9149 - 38s - loss: 2.3084 - 38s/epoch - 4ms/step
Epoch 5/10
9149/9149 - 38s - loss: 2.1847 - 38s/epoch - 4ms/step
Epoch 6/10
9149/9149 - 38s - loss: 2.0860 - 38s/epoch - 4ms/step
Epoch 7/10
9149/9149 - 38s - loss: 2.0022 - 38s/epoch - 4ms/step
Epoch 8/10
9149/9149 - 42s - loss: 1.9401 - 42s/epoch - 5ms/step
Epoch 9/10
9149/9149 - 40s - loss: 1.8895 - 40s/epoch - 4ms/step
Epoch 10/10
9149/9149 - 41s - loss: 1.8485 - 41s/epoch - 5ms/step
9149/9149 [==============================] - 22s 2ms/step
best_skor:  277422 best_gru:  8 best_unit1:  8 best_unit2:  8 best_batch 128 best_step 8
Epoch 1/10
9149/9149 - 45s - loss: 19.4802 - 45s/epoch - 5ms/step
Epoch 2/10
9149/9149 - 43s - loss: 3.1745 - 43s/epoch - 5ms/step
Epoch 3/10
9149/9149 - 43s - loss: 2.5044 - 43s/epoch - 5ms/step
Epoch

Epoch 2/10
4575/4575 - 19s - loss: 3.9330 - 19s/epoch - 4ms/step
Epoch 3/10
4575/4575 - 20s - loss: 2.6698 - 20s/epoch - 4ms/step
Epoch 4/10
4575/4575 - 18s - loss: 2.2733 - 18s/epoch - 4ms/step
Epoch 5/10
4575/4575 - 17s - loss: 2.1296 - 17s/epoch - 4ms/step
Epoch 6/10
4575/4575 - 17s - loss: 2.0528 - 17s/epoch - 4ms/step
Epoch 7/10
4575/4575 - 18s - loss: 2.0023 - 18s/epoch - 4ms/step
Epoch 8/10
4575/4575 - 18s - loss: 1.9637 - 18s/epoch - 4ms/step
Epoch 9/10
4575/4575 - 18s - loss: 1.9326 - 18s/epoch - 4ms/step
Epoch 10/10
4575/4575 - 17s - loss: 1.9058 - 17s/epoch - 4ms/step
9149/9149 [==============================] - 15s 2ms/step
Epoch 1/10
4575/4575 - 22s - loss: 17.5959 - 22s/epoch - 5ms/step
Epoch 2/10
4575/4575 - 20s - loss: 2.5394 - 20s/epoch - 4ms/step
Epoch 3/10
4575/4575 - 21s - loss: 2.1198 - 21s/epoch - 5ms/step
Epoch 4/10
4575/4575 - 21s - loss: 1.9356 - 21s/epoch - 5ms/step
Epoch 5/10
4575/4575 - 21s - loss: 1.8300 - 21s/epoch - 5ms/step
Epoch 6/10
4575/4575 - 20s - l

9149/9149 - 41s - loss: 1.7608 - 41s/epoch - 4ms/step
Epoch 6/10
9149/9149 - 41s - loss: 1.6831 - 41s/epoch - 4ms/step
Epoch 7/10
9149/9149 - 42s - loss: 1.6217 - 42s/epoch - 5ms/step
Epoch 8/10
9149/9149 - 41s - loss: 1.5680 - 41s/epoch - 5ms/step
Epoch 9/10
9149/9149 - 43s - loss: 1.5153 - 43s/epoch - 5ms/step
Epoch 10/10
9149/9149 - 45s - loss: 1.4664 - 45s/epoch - 5ms/step
9149/9149 [==============================] - 21s 2ms/step
Epoch 1/10
9149/9149 - 46s - loss: 11.8679 - 46s/epoch - 5ms/step
Epoch 2/10
9149/9149 - 43s - loss: 2.1418 - 43s/epoch - 5ms/step
Epoch 3/10
9149/9149 - 41s - loss: 1.9332 - 41s/epoch - 4ms/step
Epoch 4/10
9149/9149 - 43s - loss: 1.7993 - 43s/epoch - 5ms/step
Epoch 5/10
9149/9149 - 42s - loss: 1.6731 - 42s/epoch - 5ms/step
Epoch 6/10
9149/9149 - 45s - loss: 1.5714 - 45s/epoch - 5ms/step
Epoch 7/10
9149/9149 - 45s - loss: 1.4826 - 45s/epoch - 5ms/step
Epoch 8/10
9149/9149 - 43s - loss: 1.4094 - 43s/epoch - 5ms/step
Epoch 9/10
9149/9149 - 44s - loss: 1.3616